In [8]:
import os
import json
import subprocess
from pathlib import Path

import chromadb
from chromadb.config import Settings
import ollama


In [17]:
PROBLEMS_DIR = Path("generated_updated_problems")
SOLUTIONS_DIR = Path("generated_updated_solutions")
CHROMA_DB_DIR = "chroma_db"

TOP_K = 1
EMBED_MODEL = "qwen3-embedding:4b"
LLM_MODEL = "qwen3:4b"


In [10]:
import ollama

def get_embedding(text: str, model: str = EMBED_MODEL):
    response = ollama.embeddings(model=model, prompt=text)

    if "embedding" in response:
        emb = response["embedding"]
    elif "embeddings" in response:
        emb = response["embeddings"][0]["embedding"]
    else:
        raise ValueError(f"Unexpected Ollama response: {response}")

    if not emb or not isinstance(emb, list):
        raise ValueError("Invalid embedding")

    return emb


def generate_response(system_prompt: str, user_prompt: str):
    payload = {
        "model": LLM_MODEL,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    }

    proc = subprocess.run(
        ["ollama", "run", LLM_MODEL],
        input=json.dumps(payload).encode(),
        capture_output=True
    )

    output = proc.stdout.decode().strip()

    for line in reversed(output.splitlines()):
        try:
            obj = json.loads(line)
            if "message" in obj:
                return obj["message"]["content"]
        except:
            pass

    return output

In [11]:
from pathlib import Path
import json

PROBLEMS_DIR = Path("generated_updated_problems")

for file in sorted(PROBLEMS_DIR.glob("*.json")):
    print("----", file.name, "----")

    data = json.loads(file.read_text())
    p = data.get("Problem", data)

    print("Title:", p.get("Title"))
    print("Formal Problem Statement:", p.get("Formal Problem Statement"))
    print("Objective:", p.get("Objective"))
    print("Expected Output:", p.get("ExpectedOutput"))
    print()


---- problem_1.json ----
Title: Robust single-source regional facility location with service, capital, and labor constraints for an omnichannel retail network
Formal Problem Statement: Given a set I of candidate regional fulfillment facilities and a set J of demand nodes (stores and delivery microhubs), design a single-source omnichannel network under a set S of seasonality and disruption scenarios with probabilities p_s. For each scenario s, demand at node j is d_j^s, travel time from facility i to node j is T_ij^s, per-unit transportation cost (net of headhaul/backhaul credits) is C_ij^s, lane tender acceptance rate is A_ij^s, the ramp-up availability factor at facility i is rho_i^s, and available outbound carrier capacity at facility i is CarCap_i^s. Each facility i has steady-state throughput capacity Cap_i, capital expenditure K_i, annual fixed operating cost O_i, and zoning feasibility Z_i; each demand node j has a service-level time target SLA_j and a minimum acceptable tender a

In [12]:
def load_problem_text(path: Path):
    data = json.loads(path.read_text())
    p = data.get("Problem", data)

    return (
        f"Title: {p.get('Title','')}\n"
        f"Formal Problem Statement: {p.get('Formal Problem Statement','')}\n"
        f"Objective: {p.get('Objective','')}\n"
        f"Expected Output: {p.get('ExpectedOutput','')}"
    )


def load_solution_text(path: Path):
    return path.read_text(errors="ignore")


# -------------------------------
# BUILD CHROMA DATABASE
# -------------------------------

def build_chroma():
    client = chromadb.PersistentClient(
        path=CHROMA_DB_DIR,
        settings=Settings()
    )

    problems_col = client.get_or_create_collection("generated_updated_problems")
    solutions_col = client.get_or_create_collection("generated_updated_solutions")

    # ---- Problems ----
    for f in sorted(PROBLEMS_DIR.glob("*.json")):
        pid = f.stem.split("_")[-1]
        text = load_problem_text(f)
        emb = get_embedding(text)

        problems_col.add(
            ids=[pid],
            embeddings=[emb],
            documents=[text],
            metadatas=[{"id": int(pid)}]
        )

        print("Embedded problem", pid)

    # ---- Solutions ----
    for f in sorted(SOLUTIONS_DIR.glob("*.txt")):
        sid = f.stem.split("_")[-1]
        text = load_solution_text(f)
        emb = get_embedding(text)

        solutions_col.add(
            ids=[sid],
            embeddings=[emb],
            documents=[text],
            metadatas=[{"id": int(sid)}]
        )

        print("Embedded solution", sid)

    print("\n[✔] ChromaDB build complete.")

In [13]:
def rag_query(user_query, problems_res, solutions_res):
    context = ""

    for pid, doc in zip(problems_res["ids"][0], problems_res["documents"][0]):
        context += f"\n--- PROBLEM {pid} ---\n{doc}\n"

    for sid, doc in zip(solutions_res["ids"][0], solutions_res["documents"][0]):
        context += f"\n--- SOLUTION {sid} ---\n{doc}\n"

    system_msg = (
        "You are an expert in optimization and operations research.\n"
        "You MUST answer using only the retrieved context.\n"
        "If the context is insufficient, say so."
    )

    user_msg = f"""
USER QUERY:
{user_query}

RETRIEVED CONTEXT:
{context}
"""

    return generate_response(system_msg, user_msg)

In [14]:
def retrieve_top_k(query: str, k: int = TOP_K):
    client = chromadb.PersistentClient(path=CHROMA_DB_DIR)

    problems_col = client.get_collection("generated_updated_problems")
    solutions_col = client.get_collection("generated_updated_solutions")

    q_emb = get_embedding(query)

    problems_res = problems_col.query(
        query_embeddings=[q_emb],
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )

    solutions_res = solutions_col.query(
        query_embeddings=[q_emb],
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )

    return problems_res, solutions_res

In [19]:
def print_top_1_pair(problems_res, solutions_res):
    # Top problem
    p_id = problems_res["metadatas"][0][0]["id"]
    p_dist = problems_res["distances"][0][0]
    p_doc = problems_res["documents"][0][0]

    # Top solution
    s_id = solutions_res["metadatas"][0][0]["id"]
    s_dist = solutions_res["distances"][0][0]
    s_doc = solutions_res["documents"][0][0]

    print("\n=== TOP MATCHED PROBLEM ===")
    print(f"Problem ID: {p_id} (distance={p_dist:.4f})")
    print(p_doc)

    print("\n=== CORRESPONDING SOLUTION ===")
    print(f"Solution ID: {s_id} (distance={s_dist:.4f})")
    print(s_doc)

    print("\n" + "=" * 60 + "\n")

In [20]:
print("RAG Pipeline Loaded.")
print("1. Build Chroma")
print("2. Query")
print("3. Exit")

while True:
    choice = input("\nEnter 1/2/3: ").strip()

    if choice == "1":
        build_chroma()

    elif choice == "2":
        query = input("\nEnter your query: ")

        # Retrieve once
        problems_res, solutions_res = retrieve_top_k(query)

        # Print top-1 retrieved pair
        print_top_1_pair(problems_res, solutions_res)

        # Run RAG
        answer = rag_query(query, problems_res, solutions_res)

        print("\n=== MODEL ANSWER ===\n")
        print(answer)

    elif choice == "3":
        break

    else:
        print("Invalid choice.")

RAG Pipeline Loaded.
1. Build Chroma
2. Query
3. Exit

=== TOP MATCHED PROBLEM ===
Problem ID: 225 (distance=1.0517)
Title: Monthly Build Plan: Profit-Maximizing Product Mix with Material and Capacity Constraints
Formal Problem Statement: A mid-sized sports equipment manufacturer must decide a monthly build plan across a set of products P and plants S, subject to constrained materials M with supplier sets V_m. The decision variables are: x_{p,s}  >=  0, the production quantity of product p at plant s; y_{p,s}  in  {0,1}, a setup/changeover indicator equal to 1 if product p is produced at plant s (incurring setup time and cost); q_{m,v}  >=  0, the quantity of material m ordered from supplier v that is deliverable within the month; z_{m,v}  in  {0,1}, a binary order trigger for material m from supplier v to enforce minimum order quantities; and OT_s  >=  0, overtime hours planned at plant s. The objective is to maximize monthly profit: total sales revenue (price_p times x_{p,s} over all

KeyboardInterrupt: 